In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.datasets import mnist
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm_notebook

from util import load_data, create_dataset

In [2]:
(x_train, y_train), (x_test, y_test) = load_data()

In [3]:
def create_dataset(y_test, normal=4):
    num_normals  = int(np.sum( y_test == normal ))
    num_outliers = int(np.ceil( num_normals * 0.10 ))
    # Get non-normal elements in test dataset
    sub_arr = x_test[ y_test != normal ]
    # Generate random non-normal indices
    idx = np.random.randint(low=0, high=len(sub_arr), size=num_outliers)
    # Create contaminated test data
    x_contaminated = np.zeros((num_normals + num_outliers, 784))
    x_contaminated[num_outliers:,:] = x_test[y_test == normal]
    x_contaminated[:num_outliers,:] = sub_arr[idx]
    # Create contaminated labels
    y_contaminated = np.zeros((num_normals + num_outliers))
    y_contaminated[num_outliers:] = normal
    y_contaminated[:num_outliers] = y_test[ y_test != normal ][idx]
    # Create binary anomaly labels,   normal = 0,   outlier = 1
    binary_labels = np.copy( y_contaminated )
    binary_labels[y_contaminated == normal] = 0
    binary_labels[y_contaminated != normal] = 1
    
    return (x_contaminated, y_contaminated, binary_labels)

def create_model():
    #Sigmoid and ReLU are appropriate activation functions when having nonnegative data.
    model = keras.Sequential([
        layers.Input( shape=(784) ),
        layers.Dense(32, activation='relu' ),
        # 10 because: Rumelheadt proposed to use log2(N), Log2(784) approx 10.
        layers.Dense(10, activation='relu' ),
        layers.Dense(32, activation='relu' ),
        layers.Dense(784, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

def fit_model(model, x_train, y_train, x_test, y_test, normal=4, verbose=0):
    callbacks = [
        keras.callbacks.EarlyStopping(patience=2)
    ]
    train = x_train[y_train==normal]
    test  = x_test[y_test==normal]
    history = model.fit(train, train,
                    epochs=50,
                    batch_size=256,
                    verbose=verbose,
                    shuffle=True,
                    validation_data=(test, test),
                    callbacks=callbacks)
    return model

In [4]:
evals = np.zeros((10, 30))
for i in range(10):
    # Evaluate for all numbers
    for j in tqdm_notebook(range(30)):
        # Evaluate each method 30 times
        model = create_model()
        model = fit_model(model, x_train, y_train, x_test, y_test, normal=i)

        x, y, labels = create_dataset(y_test, normal=i)
        xhat = model.predict(x)
        err  = np.sum(np.abs(x-xhat), axis=1)
        # Max-Min normalize the error
        err /= np.max(err)
        # Compute AUC
        AUC = roc_auc_score(labels, err)
        evals[i,j] = AUC

print(np.mean(evals, axis=1))
print(np.std(evals, axis=1))

KeyboardInterrupt: 